In [1]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
from astropy.wcs import FITSFixedWarning
from astropy.wcs import WCS
import warnings
import tqdm
import os

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FITSFixedWarning)

In [2]:
def angular_distance(ra1, dec1, ra2, dec2):
    d1 = np.sin(np.radians(dec1)) * np.sin(np.radians(dec2))
    d2 = np.cos(np.radians(dec1)) * np.cos(np.radians(dec2)) * np.cos(np.radians(ra1 - ra2))
    return np.degrees(np.arccos(d1 + d2))

def max_radius(ra1, dec1, ra2, dec2):
    return np.nanmax(angular_distance(ra1, dec1, ra2, dec2))

In [3]:
ps1_folder = '/Users/zgl12/Modules/SynDiff/development/SkyCells/'
ps1_folder = '/Users/zgl12/Desktop/'
ps1_filename = 'Rings.Images.V3.fits'

# tess_file = '/Users/zgl12/Modules/SynDiff/development/TESS_FFI/TESS_FFI_Coordinates.csv'
skycell_csv = '/Users/zgl12/Modules/SynDiff/development/skycell_coordinates.csv'

tess_ffi_csv = '/Users/zgl12/Modules/SynDiff/development/TESS_FFI/TESS_FFI_Coordinates.csv'

path = ps1_folder
path = '/Users/zgl12/Modules/SynDiff/development/SkyCells/'


In [4]:
tess_ffi_df = pd.read_csv(tess_ffi_csv)

ra_center = tess_ffi_df['RA_center'].to_numpy()
dec_center = tess_ffi_df['DEC_center'].to_numpy()

In [5]:
hdul = fits.open('SkyCells/Rings.Images.V3.fits')

In [9]:
hdul[1].data

FITS_rec([(0., -90.,  29039.5,  29040., 6.9444446e-05, 6.9444446e-05, -1., 0., -0., 1., [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], 'DEC--TAN', '', 0, 0, 0., 6240, 6240, 0., 0., 0., 0., 0, 1, 0., 0., 0., 0., 0., 0., 0., 'skycell.0000.000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0., 0., 0., 0, 0, 0, 0),
          (0., -90.,  23279.5,  29040., 6.9444446e-05, 6.9444446e-05, -1., 0., -0., 1., [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], 'DEC--TAN', '', 0, 0, 0., 6240, 6240, 0., 0., 0., 0., 0, 1, 0., 0., 0., 0., 0., 0., 0., 'skycell.0000.001', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0., 0., 0., 0, 0, 0, 0),
          (0., -90.,  17519.5,  29040., 6.9444446e-05, 6.9444446e-05, -1., 0., -0., 1., [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], 'DEC--TAN', '', 0, 0, 0., 6240, 6240, 0., 0., 0., 0., 0, 1, 0., 0., 0., 0., 0., 0., 0., 'skycell.0000.002', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0., 0., 0., 0, 0, 0, 0),
          ...,
          (0.

In [5]:
if os.path.exists(skycell_csv):
    sky = True
    print('Skycell Coordinates already exist')
    new_df = pd.read_csv(skycell_csv)
else:

    hdul = fits.open(ps1_folder + ps1_filename)
    data = hdul[1].data
    header = hdul[0].header
    hdul.info()
    hdul.close()


    cols = data.columns.names
    print(cols)
    df = pd.DataFrame(columns = cols)

    new_df = pd.DataFrame(columns = ['Name', 'RA', 'DEC', 'RA_Corner1', 'DEC_Corner1', 
                                     'RA_Corner2', 'DEC_Corner2', 'RA_Corner3', 'DEC_Corner3', 
                                     'RA_Corner4', 'DEC_Corner4'])

    for j in tqdm.tqdm(range(len(data) - 63500), desc='Processing PS1 Data'):
        j += 63500

        temp = fits.Header()
        for k in ['CRVAL1','CRVAL2','CRPIX1','CRPIX2','CDELT1','CDELT2', 'PC1_1', 'PC1_2', 'PC2_1', 'PC2_2', 'NAME']:
            temp[k] = data[j][k]

        temp['NAXIS1'] = data[j]['NX']
        temp['NAXIS2'] = data[j]['NY']
        temp['RADESYS'] = 'FK5'
        temp['CTYPE1'] = 'RA---TAN'
        temp['CTYPE2'] = 'DEC--TAN'

        r_wcs = WCS(temp)

        ra, dec = r_wcs.all_pix2world(temp['NAXIS1']//2, temp['NAXIS2']//2, 0)

        fprint = r_wcs.calc_footprint()

        fpp = [fp[i] for fp in fprint for i in range(2)]

        new_df.loc[len(new_df)] = [temp['NAME'], float(ra), float(dec)] + fpp

        # break
    
    new_df.to_csv(skycell_csv, index=False)

Filename: /Users/zgl12/Desktop/Rings.Images.V3.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      11   ()      
  1  DVO_IMAGE_PS1_V2    1 BinTableHDU    171   264400R x 54C   ['D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '14E', '15A', 'A', 'J', 'J', 'E', 'I', 'I', 'E', 'E', 'E', 'E', 'I', 'I', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '121A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'J', 'J', 'J', 'J', 'I', 'I', 'I', 'I', 'E', 'E', 'E', 'I', 'I', 'J', 'J']   
['CRVAL1', 'CRVAL2', 'CRPIX1', 'CRPIX2', 'CDELT1', 'CDELT2', 'PC1_1', 'PC1_2', 'PC2_1', 'PC2_2', 'POLYTERMS', 'CTYPE', 'NPOLYTERMS', 'TZERO', 'NSTAR', 'SECZ', 'NX', 'NY', 'APMIFIT', 'DAPMIFIT', 'MCAL', 'DMCAL', 'XM', 'PHOTCODE', 'EXPTIME', 'ST', 'LAT', 'RA_CENTER', 'DEC_CENTER', 'RADIUS', 'DUMMY', 'NAME', 'DETECTION_LIMIT', 'SATURATION_LIMIT', 'CERROR', 'FWHM_X', 'FWHM_Y', 'TRATE', 'CCDNUM', 'FLAGS', 'IMAGE_ID', 'PARENT_ID', 'EXTERN_ID', 'SOURCE_ID', 'NLINK_ASTROM', 'NLINK_PHOTOM', 'DU

Processing PS1 Data: 100%|██████████| 200900/200900 [18:52<00:00, 177.37it/s]


In [6]:
new_df = pd.read_csv(skycell_csv)

for j in tqdm.tqdm(range(len(ra_center)), desc='Processing Skycells'):

    skycells = pd.DataFrame(columns = ['Name', 'Dist', 'RA', 'DEC', 'RA_Corner1', 'DEC_Corner1', 
                                     'RA_Corner2', 'DEC_Corner2', 'RA_Corner3', 'DEC_Corner3', 
                                     'RA_Corner4', 'DEC_Corner4'])

    decc = dec_center[j]
    rac = ra_center[j]

    ra_corners = np.array([tess_ffi_df.iloc[j][f'RA_corner{i}'] for i in range(1,5)])
    dec_corners = np.array([tess_ffi_df.iloc[j][f'DEC_corner{i}'] for i in range(1,5)])

    # print(ra_corners, dec_corners)

    radius_var = max_radius(ra_center[j], dec_center[j], ra_corners, dec_corners) + np.sqrt(2) * 0.4

    ccd = int(tess_ffi_df.iloc[j]['CCD'] + 4*(tess_ffi_df.iloc[j]['Camera'] - 1))
    sector = int(tess_ffi_df.iloc[j]['Sector'])

    # print(f'Processing Sector {sector} CCD {ccd}')

    radius = angular_distance(rac, decc, new_df['RA'].to_numpy(), new_df['DEC'].to_numpy())

    mask = radius < radius_var

    temp_skycells = new_df[mask]

    names = temp_skycells['Name'].to_numpy()
    ra1 = temp_skycells['RA_Corner1'].to_numpy()
    dec1 = temp_skycells['DEC_Corner1'].to_numpy()
    ra2 = temp_skycells['RA_Corner2'].to_numpy()
    dec2 = temp_skycells['DEC_Corner2'].to_numpy()
    ra3 = temp_skycells['RA_Corner3'].to_numpy()
    dec3 = temp_skycells['DEC_Corner3'].to_numpy()
    ra4 = temp_skycells['RA_Corner4'].to_numpy()
    dec4 = temp_skycells['DEC_Corner4'].to_numpy()

    skycells['Name'] = names
    skycells['Dist'] = radius[mask]
    skycells['RA'] = temp_skycells['RA'].to_numpy()
    skycells['DEC'] = temp_skycells['DEC'].to_numpy()
    skycells['RA_Corner1'] = ra1
    skycells['DEC_Corner1'] = dec1
    skycells['RA_Corner2'] = ra2
    skycells['DEC_Corner2'] = dec2
    skycells['RA_Corner3'] = ra3
    skycells['DEC_Corner3'] = dec3
    skycells['RA_Corner4'] = ra4
    skycells['DEC_Corner4'] = dec4

    skycells = skycells.reset_index(drop=True)

    if os.path.exists(path + f"Sector{str(sector).zfill(3)}/") == False:
        os.makedirs(path + f"Sector{str(sector).zfill(3)}/")

    file = path + f"Sector{str(sector).zfill(3)}/" + f"skycell_s{sector}_c{ccd}.csv"

    skycells.to_csv(file, index=False)
    # break

Processing Skycells: 100%|██████████| 1264/1264 [00:29<00:00, 43.00it/s]


In [ ]:
skycells

In [ ]:
new_df

In [ ]:

# hdul = fits.open('/Users/zgl12/Desktop/rings.v3.skycell.2247.025.stk.r.unconv.fits')
# wcs = WCS(hdul[1].header)
# # data = hdul[0].data

# for h in hdul[1].header:
#     print(h, hdul[1].header[h])
# # print(hdul[1].header['NAXIS1'], hdul[1].header['NAXIS2'])
# hdul.close()

# # print(np.shape(data))

# fp = wcs.calc_footprint()

# print(fp)

In [ ]:
# new_df[new_df['Name'] == 'skycell.2247.025']

In [ ]:
# print(data[224610])

# temp = {}
# for i in range(len(data[224610])):
#     temp[cols[i]] = data[224610][i]

# temp.pop('POLYTERMS', None)

# temp['NAXIS1'] = int(temp['NY'])
# temp['NAXIS2'] = int(temp['NX'])

# head = fits.Header(temp)
# wcs = WCS(head) 

# ra, dec = wcs.all_pix2world(temp['NY']//2, temp['NX']//2, 0)

# fprint = wcs.calc_footprint()

# fprint